In [2]:
import pandas as pd
import shutil

Documento: https://ec.europa.eu/eurostat/documents/3859598/5910701/KS-RA-09-008-EN.PDF.pdf/7ebe5561-5f2c-4458-aa9f-5bc48822f422?t=1414781566000

Importamos los datos en _csv_, **estat_urb_cecfi_en**:

In [3]:
df_csv = pd.read_csv('./input/estat_urb_cecfi_en.csv', encoding='utf8', on_bad_lines='warn')\
           .rename(columns={'TIME_PERIOD':'year','OBS_VALUE':'value','OBS_FLAG':'flag'})[['freq','indic_ur','cities','year','value','flag']]

In [36]:
df_csv.head()

,freq,indic_ur,cities,year,value,flag
0,A,EC2021V,AT,2008,300745,NaN
1,A,EC2021V,AT001C,1996,87071,NaN
2,A,EC2021V,AT001C,2001,90484,NaN
3,A,EC2021V,AT001C,2008,70660,d
4,A,EC2021V,AT002C,2008,10647,d


Importamos los datos en _tsv_:

In [37]:
ls_files = ['estat_urb_cenv','estat_urb_ceduc','estat_urb_cfermor','estat_urb_cpop1','estat_urb_cpopcb','estat_urb_cpopstr','estat_urb_ctour','estat_urb_ctran']

df_tsv = pd.concat([pd.read_csv(f'./input/{i}.tsv', sep='[,\t]', engine='python')\
                  .rename(columns={'cities\TIME_PERIOD':'cities'})\
                  .melt(id_vars=['freq','indic_ur','cities'], var_name='year', value_name='value')
                for i in ls_files])
v = df_tsv.value.replace({':':''},regex=True).str.split(" ")
df_tsv['value'] = v.str[0]
df_tsv['flag'] = v.str[1]
df_tsv = df_tsv[df_tsv.value != '']
df_tsv['value'] = df_tsv.value.astype(float)
#df = df.pivot(index=['year','cities','freq'], columns='indic_ur', values='value').reset_index()


In [38]:
df_tsv.head()

,freq,indic_ur,cities,year,value,flag
1773,A,EN2002V,AT003C,1989,6.0,
1774,A,EN2002V,AT004C,1989,93.0,
1814,A,EN2002V,DE001C,1989,31.5,
1815,A,EN2002V,DE002C,1989,30.0,
1817,A,EN2002V,DE004C,1989,27.0,


Cruzamos el CODINE

In [39]:
rel_eurostat_ine = pd.read_excel('./input/rel_eurostat_ine.ods', 'rel_Eurostat_ine', engine='odf')
df = pd.concat([df_csv, df_tsv]).join(rel_eurostat_ine.set_index('CODE'), on='cities', how='inner')

In [42]:
print(f"The DF shape: {df.shape}")
print(f"The number of cities is: {len(df.cities.unique())}")
df.head()

The DF shape: (119384, 8)
The number of cities is: 99


,freq,indic_ur,cities,year,value,flag,NAME,CODINE
22291,A,EN1002V,ES011C,1991,5.88,,Santiago de Compostela,15078
22293,A,EN1002V,ES013C,1991,4.84,,Oviedo,33044
22294,A,EN1002V,ES015C,1991,4.87,,Santander,39075
22711,A,EN1003V,ES011C,1991,19.90,,Santiago de Compostela,15078
22713,A,EN1003V,ES013C,1991,20.30,,Oviedo,33044


Unimos los DataFrames y los exportamos a _csv_:

In [43]:
df.to_csv('./output/data_eurostat.csv', index=False)

In [44]:
def prueba(x,y):
    return x + y

In [46]:
prueba(x=1,y=2,z=3)

TypeError: prueba() got an unexpected keyword argument 'z'